In [1]:
from collections import namedtuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import gym
import gym.envs.toy_text.frozen_lake
import gym.wrappers
import numpy as np
import random

In [2]:
HIDDEN_SIZE = 128 # hidden layer
BATCH_SIZE = 100
PERCENTILE = 30 # to determine performance of episode step
GAMMA = 0.9 # reward discount factor
random.seed(12345)

In [3]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    """
    one hot encode discrete inputs
    """
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n,)
        self.observation_space = gym.spaces.Box(0.0, 1.0, shape, dtype=np.float32)
        
    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [4]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [5]:
# helper classes to track episodes
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [6]:
def iterate_batches(env, net, batch_size):
    """
    generate batch of episodes
    """
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    # need to include manual softmax to convert nn output to probability distro
    # since using nn.CrossEntropyLoss which reqs raw, unnormalized data
    sm = nn.Softmax(dim=1)
    
    # env loop
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        step = EpisodeStep(observation=obs, action=action)
        obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        
        episode_steps.append(step)
        
        # if done, save and set up next
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            obs = env.reset()
            
            if len(batch) == batch_size:
                yield batch
                batch = []

In [7]:
def filter_batch(batch, percentile):
    """
    remove low performing episodes from batch for upcoming training step
    """
    # penalize rewards acquired at higher # steps (discount)
    filter_func = lambda ep: ep.reward * (GAMMA ** len(ep.steps))
    disc_rewards = list(map(filter_func, batch))
    #rewards = list(map(lambda ep: ep.reward, batch))
    
    # create bound of reward acceptability
    reward_bound = np.percentile(disc_rewards, percentile)
    
    # keeping the elite episodes to continue training on upcoming rounds
    train_obs, train_act, elite_batch = [], [], []
    
    for ep, disc_rew in zip(batch, disc_rewards):
        if disc_rew > reward_bound:
            train_obs.extend(map(lambda step: step.observation, ep.steps))
            train_act.extend(map(lambda step: step.action, ep.steps))
            elite_batch.append(ep)
        
    return elite_batch, train_obs, train_act, reward_bound

In [8]:
# train
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(is_slippery=False)
env.spec = gym.spec('FrozenLake-v0')
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)
env = DiscreteOneHotWrapper(env)
env = gym.wrappers.Monitor(env, directory='monitor', force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer = SummaryWriter(comment='-frozen-lake-0')

full_batch = [] # to accumulate elite episodes over iterations
for i, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    reward_mean = float(np.mean(list(map(lambda step: step.reward, batch))))
    full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)
    if not full_batch:
        continue
    
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts)
    full_batch = full_batch[-500:]
    
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    
    print('%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f' %
         (i, loss_v.item(), reward_mean, reward_bound))
    writer.add_scalar('loss', loss_v.item(), i)
    writer.add_scalar('reward_mean', reward_mean, i)
    writer.add_scalar('reward_bound', reward_bound, i)
    
    if reward_mean > 0.8:
        print('Solved!')
        break
        
writer.close()

0: loss=1.396, reward_mean=0.0, rw_bound=0.0
1: loss=1.386, reward_mean=0.0, rw_bound=0.0
2: loss=1.380, reward_mean=0.0, rw_bound=0.0
3: loss=1.376, reward_mean=0.0, rw_bound=0.0
4: loss=1.372, reward_mean=0.0, rw_bound=0.0
5: loss=1.366, reward_mean=0.0, rw_bound=0.0
6: loss=1.360, reward_mean=0.0, rw_bound=0.0
7: loss=1.354, reward_mean=0.0, rw_bound=0.0
8: loss=1.353, reward_mean=0.0, rw_bound=0.0
9: loss=1.348, reward_mean=0.0, rw_bound=0.0
10: loss=1.344, reward_mean=0.0, rw_bound=0.0
11: loss=1.339, reward_mean=0.0, rw_bound=0.0
12: loss=1.332, reward_mean=0.0, rw_bound=0.0
13: loss=1.328, reward_mean=0.0, rw_bound=0.0
14: loss=1.323, reward_mean=0.0, rw_bound=0.0
15: loss=1.318, reward_mean=0.0, rw_bound=0.0
16: loss=1.312, reward_mean=0.1, rw_bound=0.0
17: loss=1.309, reward_mean=0.1, rw_bound=0.0
18: loss=1.303, reward_mean=0.1, rw_bound=0.0
19: loss=1.297, reward_mean=0.0, rw_bound=0.0
20: loss=1.292, reward_mean=0.1, rw_bound=0.0
21: loss=1.289, reward_mean=0.0, rw_bound=0.